In [1]:
import sys
sys.path.append("/media/sdb1/dtoth/TensorMol")
from TensorMol import *
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

Searching for Installed Optional Packages...
Pyscf is not installed -- no ab-initio sampling


--------------------------
    ፐℇℕЅ⦾Ɽ-ᙏ⦾ℒ  0.0
--------------------------
By using this software you accept the terms of the GNU public license in 
COPYING, and agree to attribute the use of this software in publications as: 

K.Yao, J. E. Herr, J. Parkhill. TensorMol 0.0 (2016)
--------------------------
--------------------------


In [4]:
def david_AnnealHarmonic(set_ = "david_test", Anneal = True, WriteNM_ = False):
	"""
	Optionally anneals a molecule and then runs it through a finite difference normal mode analysis

	Args:
		set_: dataset from which a molecule comes
		Anneal: whether or not to perform an annealing routine before the analysis
		WriteNM_: whether or not to write normal modes to a readable file
	Returns:
		Frequencies (wavenumbers)
		Normal modes (cartesian)
	"""

	a = MSet(set_)
	a.ReadXYZ(set_)
	manager= TFMolManage("Mol_uneq_chemspider_ANI1_Sym_fc_sqdiff_BP_1" , None, False, RandomTData_=False, Trainable_=False)
	x_ = a.mols[6] #Choose index of molecule in a given dataset
	qmanager= TFMolManage("Mol_chemspider9_multipole_ANI1_Sym_Dipole_BP_1" , None, False, RandomTData_=False, Trainable_=False)
	EnergyField = lambda x: manager.Eval_BPForceSingle(Mol(x_.atoms,x),True)[0]
	ForceField = lambda x: manager.Eval_BPForceSingle(Mol(x_.atoms,x),True)
	ChargeField = lambda x: qmanager.Eval_BPDipole(Mol(x_.atoms,x),False)[2][0]
	masses = np.array(map(lambda x: ATOMICMASSESAMU[x-1],x_.atoms))
	m_ = masses
	eps_ = 0.04 #finite difference step
	if Anneal == True:
		PARAMS["OptMomentum"] = 0.0
		PARAMS["OptMomentumDecay"] = 0.9
		PARAMS["OptStepSize"] = 0.02
		PARAMS["OptMaxCycles"]=200
		PARAMS["MDdt"] = 0.2
		PARAMS["RemoveInvariant"]=True
		PARAMS["MDMaxStep"] = 100
		PARAMS["MDThermostat"] = "Nose"
		PARAMS["MDV0"] = None
		PARAMS["MDTemp"]= 1.0
		annealx_ = Annealer(ForceField, ChargeField, x_, "Anneal")
		annealx_.Prop()
		x_.coords = annealx_.Minx.copy()
		x_.WriteXYZfile("./results/", "davidIR_opt")
	HarmonicSpectra(EnergyField, x_.coords, masses, x_.atoms, WriteNM_ = True)
